## Einleitung
Question and Answer Generation (QAG) ist ein wichtiger Anwendungsfall, bei dem automatisch Fragen und Antworten zu einem gegebenen Text generiert werden.
### Use-Case
QAG hat viele praktische Anwendungen:

- Content-Erstellung: QAG kann Content-Erstellern und Autoren dabei helfen, Inhalte effizienter zu erstellen. Anstatt manuell Fragen und Antworten zu einem Text zu formulieren, kann QAG dies automatisch erledigen.
- E-Learning: In Bildungseinrichtungen und E-Learning-Plattformen kann QAG verwendet werden, um Übungen und Quizfragen zu generieren, die das Verständnis der Schüler und Lernenden fördern.
- FAQ-Generierung: Unternehmen können QAG nutzen, um FAQs für ihre Produkte oder Dienstleistungen zu erstellen. Das spart Zeit und Ressourcen bei der Pflege von Kundenunterstützungsseiten.
- Chatbots und Virtual Assistants: QAG kann dazu verwendet werden, um Chatbots und virtuellen Assistenten die Fähigkeit zu geben, auf natürliche Weise gestellte Fragen zu beantworten.
### Zielsetzung

Die Zielsetzung ist: Die Entwicklung eines QAG-Modells, das in der Lage ist, qualitativ hochwertige Fragen und Antworten auf Texte von CNN (https://huggingface.co/datasets/StellarMilk/newsqa) zu generieren.

### Github Repo
Das für das Training verwendete Repository ist https://github.com/asahi417/lm-question-generation.

## Modellarchitektur (Jan)

Um den Use- Case möglichst gut abzudecken sind wir auf das Paper von Asahi Ushio and Fernando Alva-Manchego and Jose Camacho-Collados gestossen (https://arxiv.org/pdf/2305.17002.pdf), welches unterschiedliche QAG models vergleicht. 

Das beste beschriebene Modell aufgrund von der Metrik F1/exact match, welches in der Tabelle 2 beschrieben wird ist die T5- end-end Architektur. Zusätzlich zu den besseren Metriken sind die Trainingskosten beim end-end Aufbau besser.

### End-End Ansatz
Es gibt unterschiedliche QAG-Ansätze. In diesem Bericht wird der Ansatz End2End verwendet, welcher innerhalb eines Modelles und in "einem Schritt", sowohl die Frage, wie auch die Antwort erstellt.

<img src="./images/qag.png" alt="Aufbau" width="500">

### T5- Transformer

Der T5- Transformer wird im Paper "Exploring the Limits of Transfer Learning with a Unified
Text-to-Text Transformer" (https://jmlr.org/papers/volume21/20-074/20-074.pdf) beschrieben.

Diese Modelle werden vortrainiert, um dann auf Downstream Tasks angewendet werden zu können. Dabei werden alle Aufgabenstellungen als text-to-text Problem angesehen. Dadurch könnten sehr unterschiedliche Fragestellungen abgebildet werden.

<img src="./images/T5-Aufbau.png" alt="Aufbau" width="700">

Die Modellarchitektur ist sehr ähnlich wie der ursprüngliche Transformer (https://arxiv.org/pdf/1706.03762.pdf), es wurden aber einzelne Änderungen gemacht:
- Layer Normalization: Es wird nur eine vereinfachte Layer-Normalisierung verwendet, welche die Aktivierungen umskaliert, ohne additive Bias hinzuzufügen.
- Dropout: Es wird an unterschiedlichen stellen Dropout eingesetzt.
- Embedding: Der ursprüngliche Transformer verwendent ein Sinusförmiges Embedding oder ein gelerntes Positionsembedding. In diesem Aufbau wird ein relatives Positionsembedding verwendet.

Es gibt noch weitere kleinere Anpassungen. Im grossen und ganzen entspricht es der ursprünglichen Transformerarchitektur.

Dabei verwenden wir die folgenden Modellausführungen:

#### Modellgrössen

Es wurden die Small und Base Varianten untersucht. Diese haben folgende Parameter:

| Modeltyp | Output Dimensionalität MLP pro Block | Anzahl Encoder und Decoder Layer |sub-layers and embeddings Dimension|Anzahl Parameter|
| :----:   | :---:                                | :-------: |:-------:|:-------:|
| Small    | 2048                             |    6     |512|60 * 10^6|
| Base     | 3072                                  |    12   |768|220 * 10^6|

## Trainingsprozess

Für das Trainieren des Modells wurde die Package lmqg verwendet, welche vom Repository [asahi417/lm-question-generation](https://github.com/asahi417/lm-question-generation) stammt. Jedoch hatten wir beim Trainingsprozess Probleme mit der Metrik "METEOR", welche eine JAVA Runtime benötigte und somit das Training auf dem I4DS verhinderte. Das hat uns dazu veranlasst, die Package zu forken und zu editieren. Unser Fork ist unter [swiggy123/lm-question-generation](https://github.com/swiggy123/lm-question-generation) zu finden.

Mit der LMQG CLI Schnittstelle konnte das Modell simpel trainiert werden. Wir trainierten das Modell basierend auf den Hyperparametern des ACL 2023 Papers, welche hier zu finden sind: [2023_acl_qag/model_finetuning.end2end.sh](https://github.com/asahi417/lm-question-generation/blob/master/misc/2023_acl_qag/model_finetuning.end2end.sh). Unser Trainingsprozess wurde Beispielweise beim t5-small Modell so gestartet:

```bash
lmqg-train-search -d "StellarMilk/newsqa" -m "t5-small" -b 2 -g 2 4 -c "small_trained_ckpt" -i 'paragraph' -o 'questions_answers' -p 'qag' --epoch-partial 10 -e 15 --max-length-output-eval 512 --max-length-output 512
```

Wobei die Parameter folgendes bedeuten:
|        Parameter         |                                Beschreibung                                 |   Gewählter Wert   |
| :----------------------: | :-------------------------------------------------------------------------: | :----------------: |
|            -d            |              Hugging Face Dataset, das verwendet werden soll.               | StellarMilk/newsqa |
|            -m            |                         Modell, das trainiert wird.                         |      t5-small      |
|            -b            |                                Batchgrösse.                                 |         2          |
|            -g            |                    Schritte für Gradientenakkumulation.                     |      2 oder 4      |
|            -c            |                            Pfad für Checkpoint.                             | small_trained_ckpt |
|            -i            |                         Input-Spalte des Datasets.                          |     paragraph      |
|            -o            |                         Output-Spalte des Datasets.                         | questions_answers  |
|            -p            |                              Modelltyp-Prefix.                              |        qag         |
|     --epoch-partial      |   Anzahl der Epochen, die in der ersten Trainingsphase trainiert werden.    |         10         |
|            -e            | Mindestanzahl der Epochen, die in der zweiten Trainingsphase erreicht wird. |         15         |
| --max-length-output-eval |      Maximale Sequenzlänge für die Ausgabesequenz bei der Evaluation.       |        512         |
|   --max-length-output    |                Maximale Sequenzlänge für die Ausgabesequenz.                |        512         |

Das Modell wird in einem 2-Phasen Hyperparameteroptimierungsprozess trainiert, welche in dieser Grafik dargestellt ist:

<img src="https://raw.githubusercontent.com/asahi417/lm-question-generation/master/assets/grid_search.png" alt="Trainingsprozess" width="800"><br>

In unserem Beispiel werden in der ersten Phase 12 Modelle 10 Epochen trainiert. Danach werden die besten 5 Modelle gewählt und in der zweiten Phase 5 weitere Epochen trainiert (Total 5). Falls das beste Modell bei der Epoche 15 am besten ist, wird dieses Modell weitertrainiert, bis die Validierungsmetrik (standardmässig BLEU-4) nicht mehr besser wird. Falls das beste Modell bei der Epoche 15 nicht am besten ist, wird das Modell mit der besten Metrik gewählt und nicht weitertrainiert.

### Modellliste
Hier eine Liste aller trainierten und/oder evaluierten Modelle:
|                  Modellname                   | Beschreibung                                                        | Evaluationsordner                 | Selbst trainiert? |
| :-------------------------------------------- | :------------------------------------------------------------------ | :-------------------------------- | :---------------- |
|            lmqg/t5-base-squad-qag             | T5-Baseline Modell, welches auf SQuAD trainiert wurde.              | evaluation_base/                  | Nein              |
|    StellarMilk/t5-base-newsqa-qag-trained     | T5-Baseline Modell, welches auf NewsQA trainiert wurde.             | evaluation_base_trained/          | Ja                |
|            lmqg/t5-small-squad-qag            | T5-Small Modell, welches auf SQuAD trainiert wurde.                 | evaluation_small/                 | Nein              |
|    StellarMilk/t5-small-newsqa-qag-trained    | T5-Small Modell, welches auf NewsQA trainiert wurde.                | evaluation_small_trained          | Ja                |
|   StellarMilk/t5-small-newsqa-qag-finetuned   | lmqg/t5-small-squad-qag Modell, welches auf NewsQA weitertrainiert wurde. | evaluation_small_finetuned        | Ja                |
| StellarMilk/t5-small-squad-newsqa-qag-trained | T5-Small Modell, welches auf SQuAD und NewsQA trainiert wurde.      | evaluation_small_combined_trained | Ja |

## Trainingsparameter mit Lossfunktion usw...

Während des Trainings wird auf eine Kreuzvalidierung verzichtet, da wir die Rechenkapazitäten des Slurm Cluster nicht überstrapazieren möchten. Um Fehler abschätzen zu können, wäre dies Sinnvoll.

In [1]:
import utils

log_file_path = "./slurm/logs/train_base_err.txt"
ckpt_dir = "base_trained"

name = utils.get_best_model(log_file_path, ckpt_dir=ckpt_dir)
df, average_loss = utils.get_loss(log_file_path, name)

df

,Name,Epoch,Step,Loss
0,base_trained_ckpt/model_mzgdpa,0,50,0.494186
1,base_trained_ckpt/model_mzgdpa,0,100,0.520353
2,base_trained_ckpt/model_mzgdpa,0,150,0.463166
3,base_trained_ckpt/model_mzgdpa,0,200,0.321157
4,base_trained_ckpt/model_mzgdpa,0,250,0.325742
...,...,...,...,...
119,base_trained_ckpt/model_mzgdpa,9,6000,0.277301
120,base_trained_ckpt/model_mzgdpa,9,6050,0.220752
121,base_trained_ckpt/model_mzgdpa,9,6100,0.194521
122,base_trained_ckpt/model_mzgdpa,9,6150,0.193140


## Datengrundlage (Sowohl EDA wie auch beschreiben (newsqa usw.))

## Erroranalyse (min. 3 Stück) (Florin)

Evaluation Small -> welche Fehler macht das Modell?

## Verbesserungvorschläge und dazugehörige Evaluation

## Erklärung

## Weitere Ideen

## Fazit